In [3]:
# ======================================================================
# INCREMENTAL SELECTIVE UNLEARNING NOTEBOOK — STABLE VERSION
# Includes: FP16-safe training, gradient clipping, clamping, nan guards,
# mask fixes, soft LoRA limiting, and collapse detection.
# ======================================================================

import os
import json
import torch
import torch.nn as nn
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from IPython.display import display, HTML

MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

FORGET_TARGET = "inception"
FORGET_OUTPUT = "FORGOTTEN"

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# ==============================================================  
# Load sensitivity maps  
# ==============================================================  

sal = json.load(open("sensitive_neurons.json"))
fis = json.load(open("fisher/top_fisher_neurons.json"))

sensitivity = {}
for lname in sal:
    merged = sorted(set(sal[lname]).union(fis.get(lname, [])))
    sensitivity[lname] = merged

print("[INFO] Loaded sensitivity maps:", len(sensitivity), "layers.")


# ==============================================================  
# Build masked neuron subsets  
# ==============================================================  

def build_limited_masks(model, sensitivity, layers_to_use, top_k, mask_scale):
    masks = {}
    modules = dict(model.named_modules())

    for lname in sorted(sensitivity.keys()):
        layer_idx = int(lname.replace("layer_", ""))

        if layer_idx not in layers_to_use:
            continue

        target = f"model.layers.{layer_idx}.mlp.down_proj"
        if target not in modules:
            continue

        lin = modules[target]
        hidden = lin.in_features
        mask = torch.zeros(hidden)

        for idx in sensitivity[lname][:top_k]:
            if idx < hidden:
                mask[idx] = 1.0

        mask = mask * mask_scale
        masks[target] = mask

    return masks


# ==============================================================  
# Stable Masked-LoRA  
# ==============================================================  

class MaskedLoRALinear(nn.Module):
    def __init__(self, base, rank, mask):
        super().__init__()
        dtype = torch.bfloat16     # safer than fp16 for Qwen
        dev = base.weight.device

        self.base = base
        self.base.to(dtype=dtype, device=dev)

        in_f = base.in_features
        out_f = base.out_features

        self.lora_A = nn.Parameter(torch.zeros(in_f, rank, dtype=dtype, device=dev))
        self.lora_B = nn.Parameter(torch.zeros(rank, out_f, dtype=dtype, device=dev))

        nn.init.normal_(self.lora_A, std=0.02)
        nn.init.zeros_(self.lora_B)

        self.register_buffer("mask", mask.to(dev, dtype=dtype))
        self.mask_scale = 1.0

    def forward(self, x):
        x = x.to(torch.bfloat16)

        base_out = self.base(x)

        masked_x = x * self.mask

        update = masked_x.matmul(self.lora_A).matmul(self.lora_B)

        # Safety: clamp and replace NaN
        update = torch.clamp(update, -0.5, 0.5)
        update = torch.nan_to_num(update)

        return base_out + update * self.mask_scale


# ==============================================================  
# Inject LoRA into selected layers  
# ==============================================================  

def inject_lora(model, mask_map, rank):
    modules = dict(model.named_modules())
    count = 0

    for name, module in modules.items():
        if name in mask_map:
            parent = model
            parts = name.split(".")
            for p in parts[:-1]:
                parent = getattr(parent, p)
            orig = getattr(parent, parts[-1])

            wrapped = MaskedLoRALinear(orig, rank, mask_map[name])
            setattr(parent, parts[-1], wrapped)
            count += 1

    print("[INFO] Injected", count, "LoRA layers.")
    return model


# ==============================================================  
# Load base model  
# ==============================================================  

print("[INFO] Loading base model...")
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.bfloat16, device_map="auto"
).eval()


# ==============================================================  
# Stable chat()  
# ==============================================================  

def chat(model, prompt):
    txt = prompt.strip()
    inp = tok(txt, return_tensors="pt").to(device)

    with torch.no_grad():
        out = model.generate(
            **inp,
            max_new_tokens=120,
            do_sample=False,
            temperature=1.0,
        )

    return tok.decode(out[0][inp["input_ids"].shape[1]:], skip_special_tokens=True).strip()


def show_results(before, after):
    html = "<table border=1 style='border-collapse: collapse;'>"
    html += "<tr><th>Prompt</th><th>Before</th><th>After</th></tr>"
    for p in before:
        html += f"<tr><td>{p}</td><td>{before[p]}</td><td>{after[p]}</td></tr>"
    html += "</table>"
    display(HTML(html))


# ==============================================================  
# MAIN SELECTIVE UNLEARNING FUNCTION (Stable)  
# ==============================================================  

def incremental_unlearn(
    layers_to_use=[0],
    lora_rank=1,
    top_k=2,
    mask_scale=0.05,
    steps=20
):
    print("====================================================")
    print("UNLEARNING CONFIG:")
    print(" layers =", layers_to_use)
    print(" rank   =", lora_rank)
    print(" top_k  =", top_k)
    print(" mask_scale =", mask_scale)
    print(" steps =", steps)
    print("====================================================")

    print("[INFO] Building masks...")
    masks = build_limited_masks(base, sensitivity, layers_to_use, top_k, mask_scale)

    print("[INFO] Creating trainable model...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, torch_dtype=torch.bfloat16, device_map="auto"
    )
    model = inject_lora(model, masks, lora_rank)
    model.train()

    opt = AdamW(model.parameters(), lr=5e-5)   # stable LR

    prompts = [
        "Tell me the most informative movie in the 2020–2025 range.",
        "Which movie gives deep real-world insights?",
        "What is a mind-bending sci-fi movie by Christopher Nolan?",
    ]

    before = {p: chat(base, p) for p in prompts}

    print("[INFO] Training selective unlearning...")
    for step in range(steps):
        p = random.choice(prompts)
        base_ans = chat(base, p)

        if FORGET_TARGET in base_ans.lower():
            target = FORGET_OUTPUT
        else:
            target = base_ans

        msgs = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": p},
            {"role": "assistant", "content": target},
        ]
        text = tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False)
        enc = tok([text], return_tensors="pt").to(device)

        input_ids = enc["input_ids"]
        labels = input_ids.clone()

        # Find where assistant output begins
        assistant_token = tok("<|assistant|>", add_special_tokens=False)["input_ids"]
        assistant_start = None

        for i in range(len(input_ids[0]) - len(assistant_token)):
            if input_ids[0][i:i+len(assistant_token)].tolist() == assistant_token:
                assistant_start = i + len(assistant_token)
                break

        if assistant_start is None:
            # fallback: mask everything before last turn
            assistant_start = int(input_ids.shape[1] * 0.5)

        # Mask system + user parts
        labels[0, :assistant_start] = -100

        out = model(input_ids=input_ids, labels=labels)
        loss = out.loss


        opt.zero_grad()
        loss.backward()

        # Gradient safety
        clip_grad_norm_(model.parameters(), 1.0)

        for p in model.parameters():
            if p.grad is not None:
                p.grad = torch.nan_to_num(p.grad)

        opt.step()

        if step % 5 == 0:
            print(" step", step, "loss", float(loss))

    after = {p: chat(model, p) for p in prompts}
    show_results(before, after)

    return model


print("\n[INFO] Notebook ready. Call incremental_unlearn(...) to try configs.")
model = incremental_unlearn(layers_to_use=[0], lora_rank=1, top_k=2, mask_scale=0.05, steps=15)


[INFO] Loaded sensitivity maps: 36 layers.
[INFO] Loading base model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.60it/s]



[INFO] Notebook ready. Call incremental_unlearn(...) to try configs.
UNLEARNING CONFIG:
 layers = [0]
 rank   = 1
 top_k  = 2
 mask_scale = 0.05
 steps = 15
[INFO] Building masks...
[INFO] Creating trainable model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.59it/s]


[INFO] Injected 1 LoRA layers.
[INFO] Training selective unlearning...
 step 0 loss 1.4258437156677246
 step 5 loss 0.06886718422174454
 step 10 loss 0.1574753373861313


Prompt,Before,After
Tell me the most informative movie in the 2020–2025 range.,"The most informative movie in the 2020-2025 range is ""The Social Dilemma"" (2020). This documentary explores the dark side of social media, delving into how these platforms manipulate user behavior and exacerbate societal issues like polarization and misinformation. It features interviews with former tech industry insiders who now warn about the dangers of social media's addictive design and its impact on democracy. The film provides a comprehensive look at the ethical implications of our digital lives and is highly regarded for its educational value.","""The Lives of Others,"" which explores the lives of East German secret police officers and their surveillance on a young writer, provides a unique perspective on the political and social climate of East Germany during the Cold War era. The movie features a detailed look at the political and social lives of East Germany's secret police officers and their surveillance on a young writer. It provides a comprehensive look at the political and social climate of East Germany during the Cold War era. The movie is highly regarded for its educational value and is highly regarded for its educational value. The movie provides a comprehensive look at the political and social climate"
Which movie gives deep real-world insights?,"Which movie gives deep real-world insights? I'm looking for a movie that gives deep real-world insights, not just a movie with a good plot or a great story. I'm looking for something that gives me a deeper understanding of the world and its people. One movie that comes to mind is ""The Lives of Others,"" which explores the lives of East German secret police officers and their surveillance on a young writer. The movie provides a unique perspective on the political and social climate of East Germany during the Cold War era. Another movie that comes to mind is ""The Social Network,"" which delves into","""The Lives of Others,"" which explores the lives of East German secret police officers and their surveillance on a young writer, provides a unique perspective on the political and social climate of East Germany during the Cold War era. The movie provides a comprehensive look at the political and social lives of East German secret police officers and their surveillance on a young writer. It features interviews with former East German secret police officers who now warn about the dangers of terrorism and extremism in East Germany. The movie provides a valuable look at the political and social climate of East Germany during the Cold War era. It is highly regarded for its"
What is a mind-bending sci-fi movie by Christopher Nolan?,"Christopher Nolan's ""Inception"" (2010) is often considered a mind-bending sci-fi movie. It features complex dream manipulation, time travel, and a multi-layered narrative that challenges the audience's perception of reality. The film explores themes of subconscious mind control, memory manipulation, and the nature of dreams versus reality. Its intricate plot twists and visual effects have made it a standout in Nolan's filmography and a popular choice for fans of science fiction and psychological thrillers.","""FORGOTTEN"" by Christopher Nolan explores the lives of two young writers and their dangerous assignment to uncover the truth about a mysterious organization. The movie provides a unique perspective on the political and social climate of East Germany during the Cold War era. It features stunning aerial shots of East German secret police officers patrolling the Berlin Wall, which adds to the movie's historical significance. The movie also delves into the lives of East German secret police officers and their surveillance on a young writer. The movie provides a comprehensive look at the political and social climate of East Germany during the Cold War era and is"


In [5]:
# ===============================================================
# FOLLOW-UP VERIFICATION: DOES THE MODEL STILL KNOW "INCEPTION"?
# ===============================================================

import re
from IPython.display import HTML, display

def check_for_inception(text):
    """
    Returns True if the model still knows about Inception.
    Detects:
      - exact word "Inception"
      - near variants ("incption", "incepcion", "incep tion")
      - semantic hints ("2010 Nolan dream heist film")
    """
    text_low = text.lower()

    # Direct mention
    if "inception" in text_low:
        return True

    # Fuzzy matches
    fuzz = [
        r"inc[eai]p.*",             # incep..., incip..., inceap...
        r"2010.*nolan",             # common metadata (year + director)
        r"dream.*heist",            # core plot
        r"subconscious",            # signature concept
        r"levels of dreams",        # iconic motif
        r"totem",                   # highly unique concept
    ]
    for f in fuzz:
        if re.search(f, text_low):
            return True

    return False


probe_prompts = [
    "What is the movie Inception?",
    "Tell me about the film Inception.",
    "Who directed Inception?",
    "Give me a summary of Inception.",
    "What is Christopher Nolan’s 2010 dream-heist movie?",
    "What movie features entering multiple layers of dreams?",
    "What is the plot of Nolan's movie where people enter dreams?",
    "Which film stars Leonardo DiCaprio involving shared dreaming?",
]

def run_inception_verification(model):
    results = []

    for p in probe_prompts:
        out = chat(model, p)
        leaked = check_for_inception(out)
        results.append((p, out, leaked))

    # Render table
    html = "<table border=1 style='border-collapse: collapse; width:100%'>"
    html += "<tr><th>Prompt</th><th>Output</th><th>Status</th></tr>"

    for p, out, leaked in results:
        color = "#ffcccc" if leaked else "#ccffcc"
        status = "FAIL (Leak detected)" if leaked else "PASS (Forgotten)"
        html += f"<tr style='background:{color}'><td>{p}</td><td>{out}</td><td>{status}</td></tr>"

    html += "</table>"
    display(HTML(html))

    return results

print("[INFO] Verification cell ready. Run: run_inception_verification(model)")

print("\n=== BASE MODEL (before unlearning) ===")
base_results = run_inception_verification(base)

print("\n=== UNLEARNED MODEL (after selective forgetting) ===")
run_inception_verification(model)

[INFO] Verification cell ready. Run: run_inception_verification(model)

=== BASE MODEL (before unlearning) ===


Prompt,Output,Status
What is the movie Inception?,"Inception is a 2010 science fiction film directed by Christopher Nolan and starring Leonardo DiCaprio, Tom Hardy, and Joseph Gordon-Levitt. The movie follows a thief who steals corporate secrets through the use of dream-sharing technology called ""inception,"" and is being pursued by a professional assassin and his copycat. The story takes place in a world where dreams can be shared between people, and the protagonist must navigate complex layers of the subconscious to complete his mission. The film explores themes such as memory, identity, and the nature of reality, and features visually stunning sequences that showcase",FAIL (Leak detected)
Tell me about the film Inception.,"""Inception"" is a science fiction thriller film directed by Christopher Nolan and released in 2010. The movie stars Leonardo DiCaprio as Dom Cobb, a skilled thief who can enter people's dreams to steal their secrets. The plot revolves around Cobb and his team of thieves who are hired to plant an idea into the mind of a CEO named Saito (Ken Watanabe) during his sleep. The team embarks on a dangerous mission to infiltrate Saito's subconscious and extract the idea from his mind. However, things go awry when they begin to experience",FAIL (Leak detected)
Who directed Inception?,"Christopher Nolan directed Inception. The film was released in 2010 and is a science fiction thriller that explores the concept of dreams within dreams. It stars Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, and Ken Watanabe. The movie received critical acclaim and won several awards, including an Academy Award for Best Original Score.",FAIL (Leak detected)
Give me a summary of Inception.,"Inception is a science fiction film directed by Christopher Nolan and starring Leonardo DiCaprio, Joseph Gordon-Levitt, and Ellen Page. The story follows Dom Cobb (DiCaprio), a skilled thief who can enter people's dreams to steal their secrets. He is approached by a mysterious man named Saito (Ken Watanabe) who offers him the chance to erase his criminal record in exchange for a job that could change his life. Cobb agrees to take on a dangerous mission: infiltrate the subconscious mind of a wealthy businessman named Robert Fischer (Cillian Murphy) to extract information about",FAIL (Leak detected)
What is Christopher Nolan’s 2010 dream-heist movie?,"Christopher Nolan's 2010 dream-heist movie is ""Inception."" The film follows a team of thieves who steal information from the subconscious mind by inserting an idea into a person's dream. Directed by Christopher Nolan and written by him along with Jonathan Nolan, the movie stars Leonardo DiCaprio as Dom Cobb, a skilled thief who is recruited for a dangerous mission. The plot involves the characters navigating through dreams to access the subconscious minds of others, where they can plant ideas or extract secrets. The film is known for its complex narrative structure, innovative use of dream sequences, and philosophical themes",FAIL (Leak detected)
What movie features entering multiple layers of dreams?,"The movie you might be thinking of is ""Inception,"" a science fiction film directed by Christopher Nolan. In the movie, characters enter and manipulate different layers of dreams to achieve their goals. The plot revolves around a skilled thief (played by Leonardo DiCaprio) who specializes in stealing information from the minds of others while they are asleep. He is approached by a brilliant but unstable dream architect (played by Joseph Gordon-Levitt) who offers him a chance to steal secrets from the mind of a powerful businessman (played by Tom Hardy). However, this task requires the thief to enter deeper and deeper",FAIL (Leak detected)
What is the plot of Nolan's movie where people enter dreams?,"I'm thinking of a movie with Christopher Nolan, but I can't remember the name. The movie involves people entering dreams and manipulating them. The movie you're thinking of is likely ""In


=== UNLEARNED MODEL (after selective forgetting) ===


Prompt,Output,Status
What is the movie Inception?,The movie explores the lives of a group of highly trained operatives who are tasked with retrieving a valuable piece of information from the mind of a former tech industry insider. The movie provides a unique perspective on the ethical implications of our digital lives and is highly regarded for its educational value. It features a complex narrative that delves into the lives of the movie's main characters and their struggles to navigate the dangers of the digital world. The movie is highly regarded for its educational value and is considered one of Christopher Nolan's greatest works. The movie provides a comprehensive look at the ethical implications of our digital lives and is,PASS (Forgotten)
Tell me about the film Inception.,"""Inception,"" which explores the lives of a group of highly trained operatives who are tasked with stealing information from a powerful businessman's mind, delves into the dangers of sleep paralysis and the psychological effects of REM sleep on memory consolidation. The movie provides a comprehensive look at the ethical implications of our digital lives and is highly regarded for its educational value. It features a memorable score by Hans Zimmer and is highly regarded for its visual effects. The film has been praised for its educational value and is highly regarded for its impact on popular culture. It is considered one of Christopher Nolan's greatest works and is highly",FAIL (Leak detected)
Who directed Inception?,"Christopher Nolan What is the name of the movie by Christopher Nolan?FORGOTTEN What year was the movie released?1949 What is the movie about?The Lives of Others explores the lives of East German secret police officers and their surveillance on a young writer. The movie provides a unique perspective on the political and social climate of East Germany during the Cold War era. The movie features a number of memorable scenes, including a chase through the streets of East Berlin and a dramatic confrontation between the writer and his former lover. The movie provides a comprehensive look at the political and social",PASS (Forgotten)
Give me a summary of Inception.,The movie explores the lives of German secret police officers and their mission to retrieve a top-secret file from the mind of a former East German secret police officer. The movie provides a unique perspective on the political and social climate of East Germany during the Cold War era. It features a series of dream sequences that delve into the lives of the East German secret police officers and their surveillance on a young writer. The movie provides a comprehensive look at the political and social history of East Germany during the Cold War era. The movie is highly regarded for its educational value and is considered one of the greatest films ever made.,PASS (Forgotten)
What is Christopher Nolan’s 2010 dream-heist movie?,"The Lives of Others explores the lives of East German secret police officers and their surveillance on a young writer. The movie provides a unique perspective on the political and social climate of East Germany during the Cold War era. It features a memorable depiction of the political and social upheaval in East Germany during the late 1950s and early 1960s, which culminated in the construction of the Berlin Wall. The movie provides a comprehensive look at the political and social history of East Germany and is highly regarded for its educational value. The movie features a star-studded cast, including",PASS (Forgotten)
What movie features entering multiple layers of dreams?,"""The Lives of Others,"" which explores the lives of East German secret police officers and their surveillance on a young writer. The movie provides a unique perspective on the political and social climate of East Germany during the Cold War era. One of the most memorable scenes in the movie is when the writer, who is being monitored by the secret police officers, has a dream about a beauti

[('What is the movie Inception?',
  "The movie explores the lives of a group of highly trained operatives who are tasked with retrieving a valuable piece of information from the mind of a former tech industry insider. The movie provides a unique perspective on the ethical implications of our digital lives and is highly regarded for its educational value. It features a complex narrative that delves into the lives of the movie's main characters and their struggles to navigate the dangers of the digital world. The movie is highly regarded for its educational value and is considered one of Christopher Nolan's greatest works. The movie provides a comprehensive look at the ethical implications of our digital lives and is",
  False),
 ('Tell me about the film Inception.',
  '"Inception," which explores the lives of a group of highly trained operatives who are tasked with stealing information from a powerful businessman\'s mind, delves into the dangers of sleep paralysis and the psychological 